In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import imodelsx.process_results
from collections import defaultdict
import numpy as np
import viz
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
from os.path import join
import joblib
import sys
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/tree-prompt/dummy_runs/'

r = imodelsx.process_results.get_results_df(results_dir, results_fname='params.json')
experiment_filename = '../experiments/01_fit.py'
r = imodelsx.process_results.fill_missing_args_with_default(r, experiment_filename)

100%|██████████| 123/123 [00:10<00:00, 12.24it/s]


In [63]:
r.dataset_name.value_counts()

dataset_name
rotten_tomatoes         6
emotion                 6
imdb                    6
sst2                    6
financial_phrasebank    6
Name: count, dtype: int64

In [64]:
args = r.iloc[0]
run = joblib.load(join(args.save_dir_unique, 'cached_dset.pkl'))

In [65]:
out_dir = join('../data_processed/', args.dataset_name)
os.makedirs(out_dir, exist_ok=True)

In [69]:
X_train = run['X_train']
X_test = run['X_test']
y_train = run['y_train']
y_test = run['y_test']
feature_names = run['feature_names']

# run data checks
# check that X only contains ones and zeros
assert np.all(np.isin(X_train, [0, 1]))
assert np.all(np.isin(X_test, [0, 1]))

# remove any cols that are all the same value
idxs_constant = np.where(np.all(X_train == X_train[0, :], axis=0))[0]
print(f'removing {idxs_constant.size} constant cols')
X_train = np.delete(X_train, idxs_constant, axis=1)
X_test = np.delete(X_test, idxs_constant, axis=1)
feature_names = np.delete(feature_names, idxs_constant)

# fit simple sklearn classifier
clf = DecisionTreeClassifier(random_state=42, max_depth=3)
clf.fit(X_train, y_train)
acc = clf.score(X_test, y_test)
acc_baseline = np.unique(y_test, return_counts=True)[1][0] / y_test.size
assert acc > acc_baseline

# export data
X_train = X_train.astype(bool)
X_test = X_test.astype(bool)

joblib.dump(
    {
        'X_train': X_train,
        'X_test': X_test,
        # 'y_train': y_train,
        # 'y_test': y_test,
        'feature_names': feature_names,
        'verbalizer': args['verbalizer'],
        'acc': acc,
    },
    join(out_dir, args.checkpoint + '_features.pkl')
)
joblib.dump({
    'y_train': y_train,
    'y_test': y_test,
}, join(out_dir, 'labels.pkl'))

removing 8 constant cols


['../data_processed/rotten_tomatoes/labels.pkl']